# Prepare data for training
Build SeqData's for the data available for download from SeqDatasets

# Set-up

In [1]:
# Import modules
import os
import sys
import numpy as np
import pandas as pd

import seqdatasets
import xarray as xr

from eugene import preprocess as pp

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
# Set wd
os.chdir("/cellar/users/aklie/data/datasets/deAlmeida_DrosophilaS2_UMI-STARR-seq")

In [3]:
# Set output dir
outdir_path = "training/2023_12_20/seqdatasets"

# Download data

In [32]:
# Load sdatas from seqdatasets
sdata_train = seqdatasets.deAlmeida22(dataset='train', download_dir=outdir_path, batch_size=10000).load()
sdata_val = seqdatasets.deAlmeida22(dataset='val', download_dir=outdir_path, batch_size=10000).load()
sdata_test = seqdatasets.deAlmeida22(dataset='test', download_dir=outdir_path, batch_size=10000).load()

Dataset deAlmeida22 Sequences_Train.fa has already been downloaded.
Dataset deAlmeida22 Sequences_activity_Train.txt has already been downloaded.
Zarr file found. Opening existing zarr file.
Dataset deAlmeida22 Sequences_Val.fa has already been downloaded.
Dataset deAlmeida22 Sequences_activity_Val.txt has already been downloaded.
Zarr file found. Opening existing zarr file.
Dataset deAlmeida22 Sequences_Test.fa has already been downloaded.
Dataset deAlmeida22 Sequences_activity_Test.txt has already been downloaded.
Zarr file found. Opening existing zarr file.


# Create SeqDatas

## Training

In [33]:
# Add metadata to sdatas
train_metadata = pd.read_csv(os.path.join(outdir_path, "deAlmeida22/Sequences_activity_Train.txt"), sep="\t")
train_metadata["train_val"] = True
train_meta_xr = xr.Dataset.from_dataframe(train_metadata)
train_meta_xr = train_meta_xr.drop_vars("index").rename_dims({"index": "_sequence"}).assign(_sequence=sdata_train["_sequence"])
sdata_train = sdata_train.merge(train_meta_xr)
val_metadata = pd.read_csv(os.path.join(outdir_path, "deAlmeida22/Sequences_activity_Val.txt"), sep="\t")
val_metadata["train_val"] = False
val_meta_xr = xr.Dataset.from_dataframe(val_metadata)
val_meta_xr = val_meta_xr.drop_vars("index").rename_dims({"index": "_sequence"}).assign(_sequence=sdata_val["_sequence"])
sdata_val = sdata_val.merge(val_meta_xr)

In [34]:
# Concatenate sdatas
sdata_training = xr.concat([sdata_train, sdata_val], dim="_sequence")
sdata_training

<xarray.Dataset>
Dimensions:                                  (_sequence: 442866, _length: 249)
Coordinates:
  * _sequence                                (_sequence) object 'chr2L_5587_5...
Dimensions without coordinates: _length
Data variables:
    seq                                      (_sequence, _length) |S1 b'A' .....
    Dev_log2_enrichment                      (_sequence) float64 5.712 ... 0....
    Hk_log2_enrichment                       (_sequence) float64 1.363 ... 1.007
    Dev_log2_enrichment_scaled               (_sequence) float64 3.527 ... 0....
    Hk_log2_enrichment_scaled                (_sequence) float64 0.8735 ... 0...
    Dev_log2_enrichment_quantile_normalized  (_sequence) float64 6.046 ... 0....
    Hk_log2_enrichment_quantile_normalized   (_sequence) float64 1.711 ... 1.361
    train_val                                (_sequence) bool True ... False

In [35]:
# Add in region metadata
genomic_coords = sdata_training["_sequence"].values
coords_df = pd.DataFrame([coord.split("_") for coord in genomic_coords], columns=["chrom", "start", "end", "strand", "set", "type", "region"])
for col in coords_df.columns:
    sdata_training[col] = xr.DataArray(coords_df[col].values, dims="_sequence").astype("str")
coords = [chrom + ":" + start + "-" + end for chrom, start, end in zip(sdata_training["chrom"].values, sdata_training["start"].values, sdata_training["end"].values)]
sdata_training["loci"] = xr.DataArray(coords, dims="_sequence").astype("str")
sdata_training = sdata_training.drop_vars("_sequence")
sdata_training["id"] = xr.DataArray(genomic_coords, dims="_sequence").astype("str")

In [36]:
# Verify that the training and validation sets are set
sdata_training["train_val"].to_series().value_counts()

True     402296
False     40570
Name: train_val, dtype: int64

In [37]:
# Ohe seqs
pp.ohe_seqs_sdata(sdata_training, seq_var="seq")
sdata_training["ohe_seq"] = sdata_training["ohe_seq"].transpose("_sequence", "_ohe", "length")

In [38]:
# Verify that ohe_seq has a one in each row
assert (sdata_training["ohe_seq"][0].sum(dim="_ohe") == 1).all()

In [39]:
sdata_training["type"].to_series().value_counts()

849bp        236338
None         109584
peaks         44926
DHSs          35424
enhancers     16594
Name: type, dtype: int64

# Test

In [40]:
test_metadata = pd.read_csv(os.path.join(outdir_path, "deAlmeida22/Sequences_activity_Test.txt"), sep="\t")
test_meta_xr = xr.Dataset.from_dataframe(test_metadata)
test_meta_xr = test_meta_xr.drop_vars("index").rename_dims({"index": "_sequence"}).assign(_sequence=sdata_test["_sequence"])
sdata_test = sdata_test.merge(test_meta_xr)

In [41]:
# Add in region metadata
genomic_coords = sdata_test["_sequence"].values
coords_df = pd.DataFrame([coord.split("_") for coord in genomic_coords], columns=["chrom", "start", "end", "strand", "set", "type", "region"])
for col in coords_df.columns:
    sdata_test[col] = xr.DataArray(coords_df[col].values, dims="_sequence").astype("str")
coords = [chrom + ":" + start + "-" + end for chrom, start, end in zip(sdata_test["chrom"].values, sdata_test["start"].values, sdata_test["end"].values)]
sdata_test["loci"] = xr.DataArray(coords, dims="_sequence").astype("str")
sdata_test = sdata_test.drop_vars("_sequence")
sdata_test["id"] = xr.DataArray(genomic_coords, dims="_sequence").astype("str")

In [42]:
# Ohe seqs
pp.ohe_seqs_sdata(sdata_test, seq_var="seq")
sdata_test["ohe_seq"] = sdata_test["ohe_seq"].transpose("_sequence", "_ohe", "length")

In [43]:
# Verify that ohe_seq has a one in each row
assert (sdata_test["ohe_seq"][0].sum(dim="_ohe") == 1).all()

# Save Zarr

In [44]:
sdata_training.to_zarr(f"{outdir_path}/deAlmeida22_training.zarr", mode="w")
sdata_test.to_zarr(f"{outdir_path}/deAlmeida22_test.zarr", mode="w")

In [45]:
# Delete sdatasets download
os.system(f"rm -rf {outdir_path}/deAlmeida22")

0

# gkmSVM fastas

In [46]:
sys.path.append('/cellar/users/aklie/data/datasets/PanCancerPeptile/bin/model_training')
from train import to_fasta

## Enhancers (positive class)

In [56]:
sdata_training["set"].to_series().value_counts()

peak        236338
negative    109584
Other        52018
positive     44926
Name: set, dtype: int64

In [47]:
sdata_training["type"].to_series().value_counts()

849bp        236338
None         109584
peaks         44926
DHSs          35424
enhancers     16594
Name: type, dtype: int64

In [57]:
pd.crosstab(sdata_training["set"].to_series(), sdata_training["type"].to_series())

type,849bp,DHSs,None,enhancers,peaks
set,,,,,
Other,0,35424,0,16594,0
negative,0,0,109584,0,0
peak,236338,0,0,0,0
positive,0,0,0,0,44926


In [69]:
# Load the summits
dev_enhancers = pd.read_csv("analysis/2023_12_11/geo/GSE183936_S2_dev_STARRseq_merged.peaks.txt.gz", sep="\t")
hk_enhancers = pd.read_csv("analysis/2023_12_11/geo/GSE183936_S2_hk_STARRseq_merged.peaks.txt.gz", sep="\t")
len(dev_enhancers), len(hk_enhancers)

(11658, 7062)

In [85]:
dev_enhancers = dev_enhancers[["chr", "summit"]]

In [86]:
dev_enhancers

,chr,summit
0,chr2R,17033339
1,chr3R,2902523
2,chr3L,8520775
3,chr2R,20148328
4,chr2R,12017573
...,...,...
11653,chr3R,22640161
11654,chr3LHet,2057141
11655,chr2L,19812289
11656,chr2R,18284060


In [106]:
test_df = sdata_training[["chrom", "start", "end", "set", "type", "strand"]].to_dataframe()
test_df[["start", "end"]] = test_df[["start", "end"]].astype(int)

In [111]:
from tqdm.auto import tqdm

In [107]:
# Add column as to whether the seq is in the dev set
test_df["dev"] = False
for i, row in tqdm(dev_enhancers.iterrows(), total=len(dev_enhancers)):
    test_df.loc[(test_df["chrom"] == row["chr"]) & (test_df["start"] <= row["summit"]) & (test_df["end"] >= row["summit"]), "dev"] = True
    test_df.loc[(test_df["chrom"] == row["chr"]) & (test_df["start"] <= row["summit"]) & (test_df["end"] >= row["summit"]), "summit"] = row["summit"]

In [108]:
t_df = test_df[test_df["dev"] == True]
t_df

,chrom,start,end,set,type,strand,dev
_sequence,,,,,,,
0,chr2L,5587,5835,positive,peaks,+,True
1,chr2L,5778,6026,positive,peaks,+,True
2,chr2L,14226,14474,positive,peaks,+,True
4,chr2L,34121,34369,positive,peaks,+,True
6,chr2L,47408,47656,positive,peaks,+,True
...,...,...,...,...,...,...,...
433504,chr2R,10532315,10532563,peak,849bp,-,True
433518,chr2R,10562688,10562936,peak,849bp,-,True
433519,chr2R,10562888,10563136,peak,849bp,-,True


In [109]:
t_df["type"].value_counts()

849bp    40874
peaks    22404
Name: type, dtype: int64

In [110]:
t_df["set"].value_counts()

peak        40874
positive    22404
Name: set, dtype: int64

In [133]:
sdata_training

<xarray.Dataset>
Dimensions:                                  (_sequence: 442866, _length: 249,
                                              _ohe: 4, length: 249)
Dimensions without coordinates: _sequence, _length, _ohe, length
Data variables: (12/18)
    seq                                      (_sequence, _length) |S1 b'A' .....
    Dev_log2_enrichment                      (_sequence) float64 5.712 ... 0....
    Hk_log2_enrichment                       (_sequence) float64 1.363 ... 1.007
    Dev_log2_enrichment_scaled               (_sequence) float64 3.527 ... 0....
    Hk_log2_enrichment_scaled                (_sequence) float64 0.8735 ... 0...
    Dev_log2_enrichment_quantile_normalized  (_sequence) float64 6.046 ... 0....
    ...                                       ...
    set                                      (_sequence) <U8 'positive' ... '...
    type                                     (_sequence) <U9 'peaks' ... 'None'
    region                                   (_sequence) <U6 'None' ... 'None'
    loci                                     (_sequence) <U24 'chr2L:5587-583...
    id                                       (_sequence) <U44 'chr2L_5587_583...
    ohe_seq                                  (_sequence, _ohe, length) uint8 ...

In [135]:
442866+41186

484052

In [134]:
sdata_test

<xarray.Dataset>
Dimensions:                                  (_sequence: 41186, _length: 249,
                                              _ohe: 4, length: 249)
Dimensions without coordinates: _sequence, _length, _ohe, length
Data variables: (12/17)
    seq                                      (_sequence, _length) |S1 b'A' .....
    Dev_log2_enrichment                      (_sequence) float64 3.418 ... 0....
    Hk_log2_enrichment                       (_sequence) float64 1.983 ... -1...
    Dev_log2_enrichment_scaled               (_sequence) float64 2.019 ... 0....
    Hk_log2_enrichment_scaled                (_sequence) float64 1.249 ... -0...
    Dev_log2_enrichment_quantile_normalized  (_sequence) float64 3.723 ... 0....
    ...                                       ...
    set                                      (_sequence) <U8 'positive' ... '...
    type                                     (_sequence) <U9 'peaks' ... 'None'
    region                                   (_sequence) <U6 'None' ... 'None'
    loci                                     (_sequence) <U23 'chr2R:10574736...
    id                                       (_sequence) <U43 'chr2R_10574736...
    ohe_seq                                  (_sequence, _ohe, length) uint8 ...

In [112]:
pd.crosstab(t_df["set"], t_df["type"])

type,849bp,peaks
set,,
peak,40874,0
positive,0,22404


In [141]:
sda = sdata_training.sel(_sequence=sdata_training["train_val"] == False)

In [142]:
pd.crosstab(sda["set"], sda["type"])/2

col_0,849bp,DHSs,None,enhancers,peaks
row_0,,,,,
Other,0.0,2486.0,0.0,739.0,0.0
negative,0.0,0.0,4141.0,0.0,0.0
peak,10863.0,0.0,0.0,0.0,0.0
positive,0.0,0.0,0.0,0.0,2056.0


In [147]:
pd.crosstab(sdata_test["set"], sdata_test["type"])/2

col_0,849bp,DHSs,None,enhancers,peaks
row_0,,,,,
Other,0.0,2288.0,0.0,732.0,0.0
negative,0.0,0.0,4289.0,0.0,0.0
peak,11188.0,0.0,0.0,0.0,0.0
positive,0.0,0.0,0.0,0.0,2096.0


In [122]:
(pd.crosstab(sdata_training["set"], sdata_training["type"]) + pd.crosstab(sdata_test["set"], sdata_test["type"]))/2

col_0,849bp,DHSs,None,enhancers,peaks
row_0,,,,,
Other,0.0,20000.0,0.0,9029.0,0.0
negative,0.0,0.0,59081.0,0.0,0.0
peak,129357.0,0.0,0.0,0.0,0.0
positive,0.0,0.0,0.0,0.0,24559.0


In [144]:
sdata_test

<xarray.Dataset>
Dimensions:                                  (_sequence: 41186, _length: 249,
                                              _ohe: 4, length: 249)
Dimensions without coordinates: _sequence, _length, _ohe, length
Data variables: (12/17)
    seq                                      (_sequence, _length) |S1 b'A' .....
    Dev_log2_enrichment                      (_sequence) float64 3.418 ... 0....
    Hk_log2_enrichment                       (_sequence) float64 1.983 ... -1...
    Dev_log2_enrichment_scaled               (_sequence) float64 2.019 ... 0....
    Hk_log2_enrichment_scaled                (_sequence) float64 1.249 ... -0...
    Dev_log2_enrichment_quantile_normalized  (_sequence) float64 3.723 ... 0....
    ...                                       ...
    set                                      (_sequence) <U8 'positive' ... '...
    type                                     (_sequence) <U9 'peaks' ... 'None'
    region                                   (_sequence) <U6 'None' ... 'None'
    loci                                     (_sequence) <U23 'chr2R:10574736...
    id                                       (_sequence) <U43 'chr2R_10574736...
    ohe_seq                                  (_sequence, _ohe, length) uint8 ...

In [136]:
6*24559.0

147354.0

In [129]:
(11658 + 7062)

18720

In [128]:
(11658 + 7062)*7

131040

In [53]:

# Drosophila fasta file
fasta_file = "/cellar/users/aklie/data/ref/genomes/dm3/dm3.fa"

In [62]:
# Should recognize the bedtools executable
import warnings
warnings.filterwarnings("ignore")
import os as _os
import sys as _sys
try:
    _bin_dir = _os.path.dirname(_sys.executable)
    _os.environ["PATH"] += _os.pathsep + _bin_dir
    from pybedtools import paths as _paths
    _paths._set_bedtools_path(_bin_dir)
except ImportError:
    raise ImportError(
        "Please install pybedtools (pip install pybedtools)"
    )

In [63]:
# Load the summits
dev_enhancers = pd.read_csv("analysis/2023_12_11/geo/GSE183936_S2_dev_STARRseq_merged.peaks.txt.gz", sep="\t")
hk_enhancers = pd.read_csv("analysis/2023_12_11/geo/GSE183936_S2_hk_STARRseq_merged.peaks.txt.gz", sep="\t")
len(dev_enhancers), len(hk_enhancers)

(11658, 7062)

In [305]:
# Need a 249bp window centered on the summit
dev_enhancers["start"] = dev_enhancers["summit"] - 124
dev_enhancers["end"] = dev_enhancers["summit"] + 125
dev_enhancers["length"] = dev_enhancers["end"] - dev_enhancers["start"]
hk_enhancers["start"] = hk_enhancers["summit"] - 124
hk_enhancers["end"] = hk_enhancers["summit"] + 125
hk_enhancers["length"] = hk_enhancers["end"] - hk_enhancers["start"]

In [306]:
# Create bed files
dev_enhancers_df = dev_enhancers[["chr", "start", "end", "Corrected enrichment"]]
dev_enhancers_df["name"] = dev_enhancers_df["chr"] + ":" + dev_enhancers_df["start"].astype(str) + "-" + dev_enhancers_df["end"].astype(str)
dev_enhancers_df["score"] = dev_enhancers_df["Corrected enrichment"]
dev_enhancers_df = dev_enhancers_df[["chr", "start", "end", "name", "score"]]
hk_enhancers_df = hk_enhancers[["chr", "start", "end", "Corrected enrichment"]]
hk_enhancers_df["name"] = hk_enhancers_df["chr"] + ":" + hk_enhancers_df["start"].astype(str) + "-" + hk_enhancers_df["end"].astype(str)
hk_enhancers_df["score"] = hk_enhancers_df["Corrected enrichment"]
hk_enhancers_df = hk_enhancers_df[["chr", "start", "end", "name", "score"]]

In [308]:
# Duplicate each entry for each strand (+/-)
dev_enhancers_df["strand"] = "+"
dev_enhancers_df_neg = dev_enhancers_df.copy()
dev_enhancers_df_neg["strand"] = "-"
dev_enhancers_df = pd.concat([dev_enhancers_df, dev_enhancers_df_neg])
hk_enhancers_df["strand"] = "+"
hk_enhancers_df_neg = hk_enhancers_df.copy()
hk_enhancers_df_neg["strand"] = "-"
hk_enhancers_df = pd.concat([hk_enhancers_df, hk_enhancers_df_neg])


In [309]:
# Shuffle the sequences
dev_enhancers_bed = dev_enhancers_bed.sample(frac=1, random_state=1234)
hk_enhancers_bed = hk_enhancers_bed.sample(frac=1, random_state=1234)

In [310]:
# Save as bed files
dev_enhancers_bed.to_csv("analysis/2023_12_11/geo/GSE183936_S2_dev_STARRseq_merged.peaks.bed", sep="\t", index=False, header=False)
hk_enhancers_bed.to_csv("analysis/2023_12_11/geo/GSE183936_S2_hk_STARRseq_merged.peaks.bed", sep="\t", index=False, header=False)

In [311]:
# Load these bed files into pybedtools
dev_enhancers_bed = pybedtools.BedTool("analysis/2023_12_11/geo/GSE183936_S2_dev_STARRseq_merged.peaks.bed")
hk_enhancers_bed = pybedtools.BedTool("analysis/2023_12_11/geo/GSE183936_S2_hk_STARRseq_merged.peaks.bed")

In [312]:
# Pull out sequences from fasta
dev_enhancers_fasta = dev_enhancers_bed.sequence(fi=fasta_file, s=True)
hk_enhancers_fasta = hk_enhancers_bed.sequence(fi=fasta_file, s=True)

In [314]:
# Write out fasta files
dev_enhancers_fasta.save_seqs("training/2023_12_19/seqdatasets/gkmSVM_dev_enhancers.fasta")
hk_enhancers_fasta.save_seqs("training/2023_12_19/seqdatasets/gkmSVM_hk_enhancers.fasta")

<BedTool(analysis/2023_12_11/geo/GSE183936_S2_hk_STARRseq_merged.peaks.bed)>

## Non-enhancers (negative class)

In [259]:
# Check for duplicate chrom, start and end in negatives
sdata_training_negatives = sdata_training.sel(_sequence=sdata_training["set"] == "negative")
sdata_training_negatives["loci"].to_series().duplicated().sum()

54792

In [260]:
# Shuffle the negatives
sdata_training_negatives = sdata_training_negatives.sel(_sequence=np.random.permutation(sdata_training_negatives["_sequence"].values))

# Each "loci" has two values, choose one at random
sdata_training_negatives = sdata_training_negatives.sel(_sequence=~sdata_training_negatives["loci"].to_series().duplicated(keep="first"))

# Check for duplicate chrom, start and end
sdata_training_negatives["loci"].to_series().duplicated().sum()

0

In [261]:
# Sample a set of 21463 negatives
sdata_training_negative_sample = sdata_training_negatives.sel(_sequence=np.random.choice(sdata_training_negatives["_sequence"].values, size=21463, replace=False))
sdata_training_negative_sample

<xarray.Dataset>
Dimensions:                                  (_sequence: 21463, _length: 249,
                                              length: 249, _ohe: 4)
Dimensions without coordinates: _sequence, _length, length, _ohe
Data variables: (12/18)
    seq                                      (_sequence, _length) |S1 b'C' .....
    Dev_log2_enrichment                      (_sequence) float64 -2.549 ... 1...
    Hk_log2_enrichment                       (_sequence) float64 0.691 ... -0...
    Dev_log2_enrichment_scaled               (_sequence) float64 -1.904 ... 0...
    Hk_log2_enrichment_scaled                (_sequence) float64 0.4667 ... -...
    Dev_log2_enrichment_quantile_normalized  (_sequence) float64 -2.77 ... 0....
    ...                                       ...
    set                                      (_sequence) <U8 'negative' ... '...
    type                                     (_sequence) <U9 'None' ... 'None'
    region                                   (_sequence) <U6 'None' ... 'None'
    loci                                     (_sequence) <U24 'chrX:11570801-...
    id                                       (_sequence) <U44 'chrX_11570801_...
    ohe_seq                                  (_sequence, length, _ohe) uint8 ...

In [262]:
# 
tmp = sdata_training_negative_sample["seq"].astype("U1").values
tmp = tmp.view("U" + str(tmp.shape[1])).flatten()
sdata_training_negative_sample["fasta_seq"] = xr.DataArray(tmp, dims="_sequence")

In [263]:
to_fasta(
    sdata_training_negative_sample,
    seq_var="fasta_seq",
    id_var="id",
    out_dir=outdir_path,
    prefix="gkmSVM_negatives",
)

['training/2023_12_19/seqdatasets/gkmSVM_negatives.fasta']

# DONE!

---